# Выбор локации для скважины

Требуется определить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов.   
Требуется модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль.   
Проанализировать возможную прибыль и риски техникой *Bootstrap.*

  
Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt

## Загрузка и подготовка данных

**Описание данных:**  

`id` — уникальный идентификатор скважины;  
`f0`, `f1`, `f2` — три признака точек (для нашего исследования их интерпретация нек важна, но сами признаки значимы);  
`product` — объём запасов в скважине (тыс. баррелей)

Прочитаем данные с 1-ого региона:

In [ ]:
df_geo_0 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_0.csv')

In [ ]:
df_geo_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [ ]:
df_geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_geo_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


Прочитаем данные с 2-ого региона:

In [ ]:
df_geo_1 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_1.csv')

In [ ]:
df_geo_1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [ ]:
df_geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_geo_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


Прочитаем данные с 3-ого региона:

In [ ]:
df_geo_2 = pd.read_csv('https://code.s3.yandex.net/datasets/geo_data_2.csv')

In [ ]:
df_geo_2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [ ]:
df_geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [ ]:
df_geo_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


### Вывод

- пропусков в данных нет;
- наблюдается высокая корреляция между колонками "f2" и "product", особенно во втором регионе.

## Обучение и проверка модели

Разделим данные на целевые признаки и целевые признаки, а также разделим  на обучающую и валидационную выборки в соотношении 75/25:

In [ ]:
target_geo_0 = df_geo_0['product']
features_geo_0 = df_geo_0.drop(['id', 'product'], axis=1)    
features_train_geo_0, features_valid_geo_0, target_train_geo_0, target_valid_geo_0 = train_test_split(features_geo_0, target_geo_0, test_size=0.25, random_state=12345)

In [ ]:
target_geo_1 = df_geo_1['product']
features_geo_1 = df_geo_1.drop(['id', 'product'], axis=1)    
features_train_geo_1, features_valid_geo_1, target_train_geo_1, target_valid_geo_1 = train_test_split(features_geo_1, target_geo_1, test_size=0.25, random_state=12345)

In [ ]:
target_geo_2 = df_geo_2['product']
features_geo_2 = df_geo_2.drop(['id', 'product'], axis=1)    
features_train_geo_2, features_valid_geo_2, target_train_geo_2, target_valid_geo_2 = train_test_split(features_geo_2, target_geo_2, test_size=0.25, random_state=12345)

Обучим модель Линейная регрессия для 1-го региона:

In [ ]:
model_geo_0 = LinearRegression().fit(features_train_geo_0, target_train_geo_0)
predictions_geo_0 = model_geo_0.predict(features_valid_geo_0)
predictions_geo_0_mean = predictions_geo_0.mean().round(4)
rmse_geo_0 = ((mean_squared_error(target_valid_geo_0, predictions_geo_0)) ** 0.5).round(4)
predictions_geo_0_done = pd.Series(predictions_geo_0, index=target_valid_geo_0.index)

display(f'Cредний запас предсказанного сырья в 1-ом регионе: {predictions_geo_0_mean} тыс. баррелей, RMSE модели для 1-го региона: {rmse_geo_0}')

'Cредний запас предсказанного сырья в 1-ом регионе: 92.5926 тыс. баррелей, RMSE модели для 1-го региона: 37.5794'

Обучим модель Линейная регрессия для 2-го региона:

In [ ]:
model_geo_1 = LinearRegression().fit(features_train_geo_1, target_train_geo_1)
predictions_geo_1 = model_geo_1.predict(features_valid_geo_1)
predictions_geo_1_mean = predictions_geo_1.mean().round(4)
rmse_geo_1 = ((mean_squared_error(target_valid_geo_1, predictions_geo_1)) ** 0.5).round(4)
predictions_geo_1_done = pd.Series(predictions_geo_1, index=target_valid_geo_1.index)

display(f'Cредний запас предсказанного сырья в 2-ом регионе: {predictions_geo_1_mean} тыс. баррелей, RMSE модели для 2-го региона: {rmse_geo_1}')

'Cредний запас предсказанного сырья в 2-ом регионе: 68.7285 тыс. баррелей, RMSE модели для 2-го региона: 0.8931'

Обучим модель Линейная регрессия для 3-го региона:

In [ ]:
model_geo_2 = LinearRegression().fit(features_train_geo_2, target_train_geo_2)
predictions_geo_2 = model_geo_2.predict(features_valid_geo_2)
predictions_geo_2_mean = predictions_geo_2.mean().round(4)
rmse_geo_2 = ((mean_squared_error(target_valid_geo_2, predictions_geo_2)) ** 0.5).round(4)
predictions_geo_2_done = pd.Series(predictions_geo_2, index=target_valid_geo_2.index)

display(f'Cредний запас предсказанного сырья в 3-ом регионе: {predictions_geo_2_mean} тыс. баррелей, RMSE модели для 3-го региона: {rmse_geo_2}')

'Cредний запас предсказанного сырья в 3-ом регионе: 94.965 тыс. баррелей, RMSE модели для 3-го региона: 40.0297'

### Вывод

- в регионах 1 и 3, показатель среднего запаса сырья `92.59` и `94.97` тыс. баррелей, но при RMSE равном `37.58` и `40.03`, что свидетельствует о неоднозначности показателя, неточности модели регрессии;
- в регионе 2 `68.72` тыс. баррелей, при RMSE равном `0.89`, что хорошо.

## Подготовка к расчёту прибыли

Учитывая, что:  
При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.  
Бюджет на разработку скважин в регионе — 10 млрд рублей.  
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.  
После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.  

Рассчитаем минимально необходимый объем добычи:

In [ ]:
budget = 10000000000
well_income = 450000
n_best = 200
min_volume = budget / well_income / n_best

In [ ]:
f"Необходимый объем с одной скважины для безубыточности: {min_volume:.4f} тыс. баррелей"

'Необходимый объем с одной скважины для безубыточности: 111.1111 тыс. баррелей'

Оценим среднии запасы сырья в регионых 1, 2, 3:

In [ ]:
display(f"Средний объем сырья в 1-ом регионе: {df_geo_0['product'].mean():.4f} тыс. баррелей")

'Средний объем сырья в 1-ом регионе: 92.5000 тыс. баррелей'

In [ ]:
display(f"Средний объем сырья во 2-ом регионе: {df_geo_1['product'].mean():.4f} тыс. баррелей")

'Средний объем сырья во 2-ом регионе: 68.8250 тыс. баррелей'

In [ ]:
display(f"Средний объем сырья в 3-ем регионе: {df_geo_2['product'].mean():.4f} тыс. баррелей")

'Средний объем сырья в 3-ем регионе: 95.0000 тыс. баррелей'

### Вывод

- необходимый объем с одной скважины для безубыточности: `111.11` тыс. баррелей;
- при этом данный показатель превышаем средний запас сырья в регионах, при среднем: в регионе 1 = `92.50`  тыс. баррелей , в регионе 2 `68.82` тыс. баррелей, в регионе 3 = `95.00` тыс. баррелей.

## Расчёт прибыли и рисков 

Определим функцию для расчёта прибыли:

In [ ]:
def revenue(predict, target):
    predictions_top_200 = predict.sort_values(ascending=False)
    target_top_200 = target[predictions_top_200.index][:200]
    rev = sum(target_top_200) * well_income - budget
    return rev

Определим функцию для расчёта вероятности риска и прибыли для каждого региона:

In [ ]:
def risk_det(prediction, target):
  state = np.random.RandomState(12345)
  results = []

  for i in range(1000):
      target_subsample = target.sample(n=500, replace=True, random_state=state)
      prediction_subsample = prediction[target_subsample.index]
      results.append(revenue(prediction_subsample, target_subsample))

  results = pd.Series(results)    
  results_mean = (results.mean() / 1000000)

  lower = (results.quantile(0.025) / 1000000)
  upper = (results.quantile(0.975) / 1000000)

  risk = (results < 0).mean() * 100

  answer = {'Средняя прибыль': '{0:.2f}'.format(results_mean),
              'Нижняя граница': '{0:.2f}'.format(lower),
              'Верхняя граница': '{0:.2f}'.format(upper),
              'Вероятность убытка': '{0:.2f}%'.format(risk)}

  return answer

Проведем расчеты для каждого региона:

In [ ]:
answer_reg_0 = risk_det(predictions_geo_0_done, target_valid_geo_0)

In [ ]:
answer_reg_1 = risk_det(predictions_geo_1_done, target_valid_geo_1)

In [ ]:
answer_reg_2 = risk_det(predictions_geo_2_done, target_valid_geo_2)

Выведем результаты:

In [ ]:
answer = answer_reg_0, answer_reg_1, answer_reg_2
results = pd.DataFrame(data=answer, index=('Регион 1', 'Регион 2', 'Регион 3'))
results

,Средняя прибыль,Нижняя граница,Верхняя граница,Вероятность убытка
Регион 1,425.94,-102.09,947.98,6.00%
Регион 2,515.22,68.87,931.55,1.00%
Регион 3,435.01,-128.88,969.71,6.40%


### Вывод

Исходя из моделирования регион 2 показал себя как наименее рискованым для разработки и получения сырья (вероятность убытка `1.00%`), при средней прибыли `435.01` млн. руб.